# Understand My Requirements & Architecture — Without Being a SysML Expert

**Use case:** "I have a `.sysml` file in Istari and I just want to know what's in it."

This notebook connects to Istari, navigates the system hierarchy to find a SysML v2 model,
runs SysGit extraction to produce structured data and diagrams, then demonstrates the full
version control lifecycle: updating a requirement, re-extracting, and snapshotting every
milestone — all without installing a SysML editor.

**What you'll get:**
- A walkthrough of Istari's version control: systems, configurations, snapshots, and tags
- 11 requirements (range, weight, speed, payload, safety, etc.) with target values
- 39 parts (propulsion, power, flight control, comms, payload, airframe) with attributes
- A requirements hierarchy diagram and a parts block diagram
- A model update demo: edit RangeReq (1500 → 1000 nm), re-extract, compare before/after
- 4 tagged snapshots capturing the full lifecycle: upload, extraction, update, re-extraction

**Example model:** Group 3 UAS — a notional expendable tailless flying wing UAV.

See [`example-output/`](example-output/) for what the results look like without running anything.

In [ ]:
# Setup — install SDK and connect to Istari
import sys, os
from pathlib import Path

# Install SDK if needed (e.g. in Colab)
try:
    import istari_digital_client
except ImportError:
    !pip install istari-digital-client python-dotenv -q

# Add repo root to path so we can import istari_client
repo_root = str(Path.cwd().parent.parent)
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from istari_client import get_client

client = get_client()
user = client.get_current_user()
print(f"Connected as: {user.display_name} ({user.email})")

In [ ]:
# Explore the system — navigate the Istari hierarchy
#
# Instead of jumping straight to a file, start from the system level.
# Istari organizes everything as: System -> Configuration -> Snapshot -> Files
#
# Istari links:
#   System: https://demo.istari.app/systems/c61e33e7-e14d-458e-8c82-7c8148c1a643
#   Model:  https://demo.istari.app/files/85b78395-9c09-4047-ad2a-7e2aa444b389

SYSTEM_ID = "c61e33e7-e14d-458e-8c82-7c8148c1a643"  # Example: Explore SysML Model
CONFIG_ID = "0af397c7-bf90-4101-befb-3f7c6a118628"  # Baseline configuration

system = client.get_system(SYSTEM_ID)
print(f"System: {system.name}")
print(f"  {system.description}\n")

# Walk the hierarchy: configurations -> tracked files -> snapshots -> files
configs = client.list_system_configurations(SYSTEM_ID, page=1, size=50)
print(f"Configurations ({configs.total}):\n")

for config in configs.items:
    print(f"  {config.name}")
    print(f"    Config ID: {config.id}")

    # What files does this configuration track?
    tracked = client.list_tracked_files(config.id, page=1, size=50)
    for tf in tracked.items:
        mode = tf.specifier_type.value  # "Latest" or "Locked"
        print(f"    Tracked file: {tf.file_id} ({mode})")

    # What snapshots exist? (point-in-time captures)
    snapshots = client.list_snapshots(configuration_id=config.id, page=1, size=10)
    print(f"    Snapshots ({snapshots.total}):")

    for snap in snapshots.items:
        tags = client.list_tags(snapshot_id=snap.id, page=1, size=10)
        tag_names = [t.tag for t in tags.items]
        tag_str = f"  [{', '.join(tag_names)}]" if tag_names else ""
        revs = client.list_snapshot_revisions(snap.id, page=1, size=50)
        print(f"      {snap.id[:8]}...{tag_str}  ({revs.total} file(s))")
        for r in revs.items:
            size_kb = r.size / 1024
            print(f"        - {r.name} ({size_kb:.1f} KB)")

In [ ]:
# Load the model and show its revision history
#
# Every time a file is updated via the SDK, Istari creates a new revision.
# The configuration tracks the file as "Latest" — so it always points to the
# newest revision. You can also lock to a specific revision for reproducibility.

MODEL_ID = "85b78395-9c09-4047-ad2a-7e2aa444b389"  # Group3 UAS Requirements

model = client.get_model(MODEL_ID)
print(f"Model: {model.display_name or 'Group3 UAS Requirements'}")
print(f"  File ID: {model.file.id}")
print(f"  Revisions: {len(model.file.revisions)}")
for rev in model.file.revisions:
    print(f"    - {rev.name} ({rev.size:,} bytes)")
print(f"  Existing artifacts: {len(model.artifacts)}")

# Show how the model connects to the system hierarchy
print(f"\nThis file is tracked in the '{configs.items[0].name}' configuration")
print(f"as a LATEST tracked file — any new revisions will be automatically picked up.")

In [ ]:
# Peek at the raw SysML text
#
# This is what the file looks like — SysML v2 syntax with requirements, parts,
# constraints, and satisfy statements all interleaved. 596 lines of text.

content = model.read_text()
lines = content.splitlines()
print(f"Total: {len(lines)} lines, {len(content):,} characters\n")
print("--- First 40 lines ---")
for i, line in enumerate(lines[:40], 1):
    print(f"{i:4d} | {line}")

# You'll see the package declaration, requirement definitions with attributes,
# and part definitions — but it's hard to quickly answer "what requirements exist?"
# or "what are the key parts?" from raw text. That's what extraction is for.

In [ ]:
# Run SysGit extraction
#
# Submits a job that parses the SysML file and produces 4 artifacts:
#   - output_requirements.json   (structured requirement data)
#   - output_parts.json          (structured parts data)
#   - requirements_hierarchy.png (visual requirements tree)
#   - parts_diagram.png          (visual parts block diagram)
#
# Takes ~1-3 minutes depending on agent availability.
# If artifacts already exist from a previous run, you can skip this cell.

from time import sleep
from datetime import datetime
from istari_digital_client import JobStatusName

print("Submitting extraction job...")
job = client.add_job(
    model_id=MODEL_ID,
    function="@istari:extract_sysmlv2",
    tool_name="sysgit",
    tool_version="0.1.8",
    operating_system="Ubuntu 22.04",
    parameters={},
)
print(f"Job: {job.id}")

# Poll until done
while True:
    sleep(5)
    job = client.get_job(job.id)
    ts = datetime.now().strftime("%H:%M:%S")
    status = job.status.name.value
    print(f"  [{ts}] {status}")
    if job.status.name in {JobStatusName.COMPLETED, JobStatusName.FAILED}:
        break

if job.status.name == JobStatusName.COMPLETED:
    print("\nExtraction complete!")
    model = client.get_model(MODEL_ID)  # refresh to see new artifacts
    print(f"Artifacts ({len(model.artifacts)}):")
    for a in model.artifacts:
        rev = a.file.revisions[0] if a.file.revisions else None
        size = f"{rev.size:,} bytes" if rev else "?"
        print(f"  - {rev.name if rev else a.name} ({size})")
else:
    print("\nExtraction failed!")
    if job.status_history:
        for s in job.status_history:
            print(f"  {s.name}: {getattr(s, 'message', '')}")

In [ ]:
# Snapshot the extraction results
#
# A snapshot captures the exact state of all tracked files at this moment.
# Think of it like a git tag — an immutable bookmark you can always return to.
#
# Before extraction: 1 file  (just the .sysml)
# After extraction:  5 files (.sysml + 4 artifacts)
#
# Creating a snapshot here means anyone can later see exactly what was in the
# system at the time we ran extraction.

from istari_digital_client import NewSnapshot, NewSnapshotTag

print("Creating post-extraction snapshot...")
snap_response = client.create_snapshot(CONFIG_ID, NewSnapshot())
snapshot = snap_response.actual_instance

# Handle both new snapshot and no-op (if snapshot already exists for this state)
if hasattr(snapshot, "id"):
    snapshot_id = snapshot.id
    print(f"Snapshot created: {snapshot_id}")
else:
    # No changes since last snapshot — find the most recent one
    snaps = client.list_snapshots(configuration_id=CONFIG_ID, page=1, size=1)
    snapshot_id = snaps.items[0].id
    print(f"No changes since last snapshot — using: {snapshot_id}")

# Tag it with a human-readable name
tag = client.create_tag(snapshot_id, NewSnapshotTag(tag="post-extraction"))
print(f"Tagged as: '{tag.tag}'")

# Verify what's in the snapshot
revs = client.list_snapshot_revisions(snapshot_id, page=1, size=50)
print(f"\nSnapshot contains {revs.total} file(s):")
for r in revs.items:
    size_kb = r.size / 1024
    print(f"  - {r.name} ({size_kb:.1f} KB)")

In [ ]:
# View requirements
#
# The extraction produces a dict keyed by qualified SysML name.
# Each requirement has: name, description, parent reference, and attributes
# (target values, units, priority).
#
# Expected: 11 requirements including RangeReq (1500 nm), MaxStructureWeight (275 lb),
# CruiseSpeed (100 knots), PayloadCapacity (125 lb), FailSafeReq, etc.

import json
from IPython.display import HTML, display

# Refresh model if needed (in case you skipped the extraction cell)
model = client.get_model(MODEL_ID)

# Find the requirements artifact
reqs_artifact = None
for a in model.artifacts:
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev and "requirements" in rev.name and rev.name.endswith(".json"):
        reqs_artifact = a
        break

if reqs_artifact:
    reqs_data = json.loads(reqs_artifact.read_text())
    print(f"Found {len(reqs_data)} requirements\n")
    
    # Build HTML table from the dict
    rows = ""
    for qname, r in reqs_data.items():
        name = r.get("name", "\u2014")
        desc = r.get("description", "\u2014")
        attrs = r.get("attributes", {})
        
        # Format attributes
        attr_parts = []
        for k, v in attrs.items():
            if k == "description":
                continue
            attr_parts.append(f"{k}: {v}")
        attr_str = ", ".join(attr_parts) if attr_parts else "\u2014"
        
        # Get the shall statement from attributes if present
        shall = attrs.get("description", desc)
        
        rows += f'<tr><td><b>{name}</b></td><td>{shall}</td><td>{attr_str}</td></tr>\n'
    
    html = f"""
    <table style="border-collapse: collapse; width: 100%; font-size: 13px;">
    <thead>
        <tr style="background: #f1f5f9;">
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1; white-space: nowrap;">Requirement</th>
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1;">Shall Statement</th>
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1;">Values</th>
        </tr>
    </thead>
    <tbody>{rows}</tbody>
    </table>
    """
    display(HTML(html))
else:
    print("Requirements artifact not found. Run the extraction cell first.")

In [ ]:
# View parts
#
# The parts JSON is a dict keyed by qualified name. Each part has:
#   - declared_name: short name (e.g. "engine", "gps", "wing")
#   - attributes: dict of name → {name, value, code, type}
#
# Expected: 39 parts across 6 subsystems (Propulsion, Power, Flight Control,
# Communication, Payload, Airframe)

parts_artifact = None
for a in model.artifacts:
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev and "parts" in rev.name and rev.name.endswith(".json"):
        parts_artifact = a
        break

if parts_artifact:
    parts_data = json.loads(parts_artifact.read_text())
    print(f"Found {len(parts_data)} parts\n")
    
    rows = ""
    for qname, p in parts_data.items():
        name = p.get("declared_name", qname.split("::")[-1])
        attrs = p.get("attributes", {})
        
        # Show attributes with values (skip unset ones for readability)
        attr_parts = []
        for aname, ainfo in list(attrs.items())[:6]:
            val = ainfo.get("value")
            if val is not None:
                attr_parts.append(f"{aname}: {val}")
            else:
                attr_parts.append(f"{aname}: <i>unset</i>")
        attr_str = "<br>".join(attr_parts)
        if len(attrs) > 6:
            attr_str += f"<br><i>+{len(attrs) - 6} more</i>"
        
        # Indent sub-parts based on :: depth
        depth = qname.count("::") - 1
        indent = "&nbsp;&nbsp;&nbsp;&nbsp;" * max(0, depth - 1)
        
        rows += f'<tr><td>{indent}<b>{name}</b></td><td>{attr_str or "—"}</td></tr>\n'
    
    html = f"""
    <table style="border-collapse: collapse; width: 100%; font-size: 13px;">
    <thead>
        <tr style="background: #f1f5f9;">
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1; white-space: nowrap;">Part</th>
            <th style="padding: 8px; text-align: left; border-bottom: 2px solid #cbd5e1;">Attributes</th>
        </tr>
    </thead>
    <tbody>{rows}</tbody>
    </table>
    """
    display(HTML(html))
else:
    print("Parts artifact not found. Run the extraction cell first.")

In [ ]:
# View diagrams
#
# requirements_hierarchy.png — tree showing TopLevelRequirements branching to
#   10 child requirements (RangeReq, MaxStructureWeight, CruiseSpeed, etc.)
#
# parts_diagram.png — block diagram showing the Drone system with 6 subsystems
#   and all 39 parts with their key attribute values

from IPython.display import Image, display, Markdown

for label, keyword in [("Requirements Hierarchy", "requirements"), ("Parts Diagram", "parts")]:
    png_artifact = None
    for a in model.artifacts:
        rev = a.file.revisions[0] if a.file.revisions else None
        if rev and keyword in rev.name and rev.name.endswith(".png"):
            png_artifact = a
            break
    
    if png_artifact:
        display(Markdown(f"### {label}"))
        img_bytes = png_artifact.read_bytes()
        display(Image(data=img_bytes))
    else:
        print(f"{label}: not found")

## Update the Model — Version Control in Action

So far, we've uploaded a model and extracted structured data. But what happens when a
requirement changes?

The range requirement is currently **1,500 nm**. Let's reduce it to **1,000 nm** and see
Istari's version control track the change through the full lifecycle:

1. **Download** the current model text from Istari
2. **Modify** the RangeReq (1500 → 1000) — just the range requirement, not ControlRange
3. **Re-upload** as a new revision with a descriptive version name
4. **Snapshot** to bookmark the updated model state
5. **Re-extract** to regenerate diagrams and structured data
6. **Snapshot** again to bookmark the new extraction results

In [ ]:
# Update the model — change RangeReq from 1500 nm to 1000 nm
#
# This is the key version control moment: we modify a tracked file through the SDK,
# and Istari automatically creates a new file revision (revision 2) on the same file.
# The "Baseline" configuration's LATEST tracking means it instantly sees the update.

import tempfile
from pathlib import Path

# Step 1: Download current model text
content = model.read_text()
print(f"Downloaded model: {len(content):,} characters")

# Step 2: Targeted edit — only change the RangeReq, not the ControlRange
#
# The model has "1500" in two requirements:
#   RangeReq     — "at least 1500 nm"  + targetValue 1500.0 (unit = "nm")  ← CHANGE
#   ControlRange — "up to 1500m range" + minRange 1500.0    (unit = "m")   ← LEAVE ALONE
#
# We use the surrounding context (unit = "nm") to avoid hitting ControlRange.

old_desc = 'attribute description = "The drone shall achieve a range of at least 1500 nm"'
new_desc = 'attribute description = "The drone shall achieve a range of at least 1000 nm"'

old_val = 'attribute targetValue : Real = 1500.0;\n            attribute unit = "nm"'
new_val = 'attribute targetValue : Real = 1000.0;\n            attribute unit = "nm"'

modified = content.replace(old_desc, new_desc).replace(old_val, new_val)

# Verify: RangeReq changed, ControlRange untouched
assert modified.count("at least 1000 nm") == 1, "Description not updated"
assert modified.count("targetValue : Real = 1000.0") == 1, "Value not updated"
assert modified.count("Control signals shall work up to 1500m range") == 1, "ControlRange should be unchanged"
print("Verified: RangeReq updated (1500 → 1000 nm), ControlRange unchanged (1500 m)")

# Step 3: Re-upload as a new revision
with tempfile.NamedTemporaryFile(suffix=".sysml", mode="w", delete=False) as tmp:
    tmp.write(modified)
    tmp_path = Path(tmp.name)

updated_model = client.update_model(
    model_id=MODEL_ID,
    path=tmp_path,
    version_name="v2 — Reduced Range",
    description="RangeReq: 1500 nm → 1000 nm",
)
tmp_path.unlink()

# Show the new revision history
model = client.get_model(MODEL_ID)
print(f"\nModel now has {len(model.file.revisions)} revisions:")
for rev in model.file.revisions:
    print(f"  - {rev.name} ({rev.size:,} bytes)")

In [ ]:
# Snapshot the update, re-extract, snapshot again
#
# This creates two new snapshots:
#   3. "post-update"        — model at revision 2, but extraction artifacts still from rev 1
#   4. "post-re-extraction" — model at revision 2, with NEW extraction artifacts
#
# The difference between snapshot 3 and 4 shows Istari tracking both the source model
# and its derived artifacts — you always know which outputs match which inputs.

from istari_digital_client import NewSnapshot, NewSnapshotTag, JobStatusName
from time import sleep
from datetime import datetime

# --- Snapshot 3: post-update ---
print("Creating post-update snapshot...")
snap_response = client.create_snapshot(CONFIG_ID, NewSnapshot())
snapshot = snap_response.actual_instance
if hasattr(snapshot, "id"):
    post_update_snap_id = snapshot.id
    print(f"  Snapshot created: {post_update_snap_id}")
else:
    snaps = client.list_snapshots(configuration_id=CONFIG_ID, page=1, size=1)
    post_update_snap_id = snaps.items[0].id
    print(f"  No changes since last snapshot — using: {post_update_snap_id}")

tag = client.create_tag(post_update_snap_id, NewSnapshotTag(tag="post-update"))
print(f"  Tagged as: '{tag.tag}'")

revs = client.list_snapshot_revisions(post_update_snap_id, page=1, size=50)
print(f"  Contains {revs.total} file(s)")

# --- Re-extract with updated model ---
print("\nSubmitting re-extraction job...")
job = client.add_job(
    model_id=MODEL_ID,
    function="@istari:extract_sysmlv2",
    tool_name="sysgit",
    tool_version="0.1.8",
    operating_system="Ubuntu 22.04",
    parameters={},
)
print(f"  Job: {job.id}")

while True:
    sleep(5)
    job = client.get_job(job.id)
    ts = datetime.now().strftime("%H:%M:%S")
    status = job.status.name.value
    print(f"  [{ts}] {status}")
    if job.status.name in {JobStatusName.COMPLETED, JobStatusName.FAILED}:
        break

if job.status.name == JobStatusName.COMPLETED:
    print("\nRe-extraction complete!")
    model = client.get_model(MODEL_ID)  # refresh
else:
    print("\nRe-extraction failed!")

# --- Snapshot 4: post-re-extraction ---
print("\nCreating post-re-extraction snapshot...")
snap_response = client.create_snapshot(CONFIG_ID, NewSnapshot())
snapshot = snap_response.actual_instance
if hasattr(snapshot, "id"):
    post_reextract_snap_id = snapshot.id
    print(f"  Snapshot created: {post_reextract_snap_id}")
else:
    snaps = client.list_snapshots(configuration_id=CONFIG_ID, page=1, size=1)
    post_reextract_snap_id = snaps.items[0].id
    print(f"  No changes since last snapshot — using: {post_reextract_snap_id}")

tag = client.create_tag(post_reextract_snap_id, NewSnapshotTag(tag="post-re-extraction"))
print(f"  Tagged as: '{tag.tag}'")

revs = client.list_snapshot_revisions(post_reextract_snap_id, page=1, size=50)
print(f"  Contains {revs.total} file(s):")

In [ ]:
# Compare: what changed after the update?
#
# Re-read the extracted requirements and show that RangeReq is now 1000 nm.
# This proves the full round-trip: edit SysML → re-upload → re-extract → see the change.

import json
from IPython.display import HTML, display

model = client.get_model(MODEL_ID)  # refresh to get latest artifacts

# Find the requirements artifact (most recent)
reqs_artifact = None
for a in reversed(model.artifacts):
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev and "requirements" in rev.name and rev.name.endswith(".json"):
        reqs_artifact = a
        break

if reqs_artifact:
    reqs_data = json.loads(reqs_artifact.read_text())
    
    # Find RangeReq and ControlRange to compare
    range_req = control_range = None
    for qname, r in reqs_data.items():
        if r.get("name") == "RangeReq":
            range_req = r
        if r.get("name") == "ControlRange":
            control_range = r
    
    if range_req:
        attrs = range_req.get("attributes", {})
        print("RangeReq after update:")
        print(f"  Description: {attrs.get('description', '—')}")
        print(f"  Target value: {attrs.get('targetValue', '—')} {attrs.get('unit', '')}")
    
    if control_range:
        attrs = control_range.get("attributes", {})
        print(f"\nControlRange (should be unchanged):")
        print(f"  Description: {attrs.get('description', '—')}")
        print(f"  Min range: {attrs.get('minRange', '—')} {attrs.get('unit', '')}")
    
    # Before/after comparison table
    html = """
    <h3>Before vs After</h3>
    <table style="border-collapse: collapse; font-size: 14px; margin-top: 12px;">
    <thead>
        <tr style="background: #f1f5f9;">
            <th style="padding: 8px 16px; border-bottom: 2px solid #cbd5e1;">Requirement</th>
            <th style="padding: 8px 16px; border-bottom: 2px solid #cbd5e1;">Before (v1)</th>
            <th style="padding: 8px 16px; border-bottom: 2px solid #cbd5e1;">After (v2)</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="padding: 8px 16px;"><b>RangeReq</b> (flight range)</td>
            <td style="padding: 8px 16px; color: #dc2626; text-decoration: line-through;">1,500 nm</td>
            <td style="padding: 8px 16px; color: #16a34a; font-weight: bold;">1,000 nm</td>
        </tr>
        <tr style="background: #f8fafc;">
            <td style="padding: 8px 16px;">ControlRange (radio range)</td>
            <td style="padding: 8px 16px;">1,500 m</td>
            <td style="padding: 8px 16px;">1,500 m (unchanged)</td>
        </tr>
    </tbody>
    </table>
    """
    display(HTML(html))
    
    print("\nThe re-extraction produced new artifacts reflecting the updated model.")
    print("Both the old and new states are preserved in tagged snapshots.")
else:
    print("Requirements artifact not found. Run the re-extraction cell first.")

In [ ]:
# View the full system state — everything Istari tracked
#
# This is the "outer loop" view. The system now has four tagged snapshots:
#   1. "initial-upload, baseline" — just the .sysml file (revision 1)
#   2. "post-extraction"         — .sysml rev 1 + 4 extraction artifacts
#   3. "post-update"             — .sysml rev 2 + artifacts from rev 1 (stale)
#   4. "post-re-extraction"      — .sysml rev 2 + 4 NEW extraction artifacts
#
# Anyone on the team can see exactly what changed and when.

print(f"System: {system.name}\n")

all_snapshots = client.list_snapshots(configuration_id=CONFIG_ID, page=1, size=20)
print(f"Total snapshots: {all_snapshots.total}\n")

for i, snap in enumerate(all_snapshots.items, 1):
    tags = client.list_tags(snapshot_id=snap.id, page=1, size=10)
    tag_names = [t.tag for t in tags.items]
    tag_str = ", ".join(tag_names) if tag_names else "untagged"
    
    revs = client.list_snapshot_revisions(snap.id, page=1, size=50)
    
    print(f"  Snapshot {i}: [{tag_str}]")
    print(f"    Created: {snap.created}")
    print(f"    Files ({revs.total}):")
    for r in revs.items:
        size_kb = r.size / 1024
        if size_kb > 1024:
            size_str = f"{size_kb/1024:.1f} MB"
        else:
            size_str = f"{size_kb:.1f} KB"
        print(f"      - {r.name:40s} {size_str:>10s}")
    print()

print("Every file, every version, every milestone — tracked in one place.")
print("Notice: snapshot 3 has the updated model but old extraction artifacts,")
print("while snapshot 4 has both the updated model and fresh extraction results.")

## Summary

From a single `.sysml` file (595 lines of SysML v2 text), the extraction produced:

| Artifact | Contents |
|----------|----------|
| `output_requirements.json` (5.5 KB) | 11 requirements — range, weight, speed, payload, temperature, safety, position, video, control |
| `output_parts.json` (34.6 KB) | 39 parts across 6 subsystems — each with typed attributes and values |
| `requirements_hierarchy.png` | Visual tree: TopLevelRequirements → 10 child requirements |
| `parts_diagram.png` | Block diagram: Drone → Propulsion, Power, Flight Control, Comms, Payload, Airframe |

### Key requirements found:

| Requirement | v1 Target | v2 Target |
|-------------|-----------|-----------|
| Range | ~~1,500 nm~~ | **1,000 nm** (updated) |
| Max Structure Weight | 275 lb | 275 lb |
| Cruise Speed | 100 knots | 100 knots |
| Payload Capacity | 125 lb | 125 lb |
| Operating Temperature | -10°C to 45°C | -10°C to 45°C |
| Position Accuracy | 2.5 m | 2.5 m |

### Version control in action

Istari tracked the full lifecycle automatically:

| Snapshot | Tag | What Happened | Files |
|----------|-----|---------------|-------|
| 1 | initial-upload, baseline | Uploaded .sysml file | 1 (model rev 1) |
| 2 | post-extraction | Ran SysGit extraction | 5 (model rev 1 + 4 artifacts) |
| 3 | post-update | Updated RangeReq: 1500 → 1000 nm | 1 (model rev 2) |
| 4 | post-re-extraction | Re-ran SysGit extraction | 5 (model rev 2 + 4 new artifacts) |

The key insight: **snapshot 3 vs snapshot 4** shows the difference between updating a model
and re-generating its derived artifacts. Istari tracks both the source model and its
extraction outputs, so you always know which artifacts match which model revision.

| What Istari Did | How |
|----------------|-----|
| Stored the .sysml file | `add_model()` → file revision 1 |
| Organized it in a configuration | "Baseline" config with LATEST tracking |
| Bookmarked the upload | Snapshot tagged "initial-upload" |
| Ran SysGit extraction | `add_job()` → 4 artifacts produced |
| Bookmarked the extraction | Snapshot tagged "post-extraction" |
| Updated a requirement (1500 → 1000 nm) | `update_model()` → file revision 2 |
| Bookmarked the update | Snapshot tagged "post-update" |
| Re-ran SysGit extraction | `add_job()` → 4 new artifacts produced |
| Bookmarked the re-extraction | Snapshot tagged "post-re-extraction" |

Anyone on the team can see what's in each snapshot, compare them, or roll back.
This is the **outer loop** — the version control layer that sits above any individual tool.

## What's Next?

- **Edit further requirements** using the update workflow above, or use the CLI: [`sysgit/`](../../sysgit/)
- **Run an nTop wing design** using these requirements — see [`use-cases/run-ntop-wing-design/`](../run-ntop-wing-design/)
- **Check design meets requirements** — cross-check CAD against SysML — see [`use-cases/check-design-meets-requirements/`](../check-design-meets-requirements/)
- **Upload a new version** of the .sysml — see [`getting-started/02_version_model.py`](../../getting-started/02_version_model.py)
- **Share with your team** — see [`getting-started/03_share_resources.py`](../../getting-started/03_share_resources.py)